# Exercise 3: Multifield and nonlinear problems

## Contents

We will learn

 - How to solve nonlinear multi-field PDEs in Gridap

## Problem statement

We consider the lid-driven cavity for the incompressible Navier-Stokes equations:

$$
\left\lbrace
\begin{aligned}
-\Delta u + \mathit{Re}\ (u\cdot \nabla)\ u + \nabla p = 0 &\text{ in }\Omega,\\
\nabla\cdot u = 0 &\text{ in } \Omega,\\
u = g &\text{ on } \partial\Omega,
\end{aligned}
\right.
$$

with $\Omega \doteq (0,1)^d$, $d=2$, and $\mathit{Re} = 10$.

$g$ is $(1,0)^t$ on the top side of the cavity and $g=0$ elsewhere.

For well-posedness, the mean value of the pressure must be zero:

$$
\int_\Omega q \ {\rm d}\Omega = 0.
$$

## Numerical Scheme

We choose inf-sub stable $Q_k/P_{k-1}$ FEs, with continuous velocities and discontinuous pressures:

The weak form reads:

Find $(u,p)\in U_h \times Q_0$ such that $[r(u,p)](v,q)=0$ for all $(v,q)\in V_0 \times Q_0$
where

$$
[r(u,p)](v,q) \doteq a((u,p),(v,q))+ [c(u)](v),
$$
with
$$
\begin{aligned}
a((u,p),(v,q)) &\doteq \int_{\Omega} \nabla v \cdot \nabla u \ {\rm d}\Omega - \int_{\Omega} (\nabla\cdot v) \ p \ {\rm d}\Omega + \int_{\Omega} q \ (\nabla \cdot u) \ {\rm d}\Omega,\\
[c(u)](v) &\doteq \int_{\Omega} v \cdot \left( Re \ (u\cdot\nabla)\ u \right)\ {\rm d}\Omega.\\
\end{aligned}
$$

We will solve this nonlinear weak equation with a Newton-Raphson method.

The Jacobian will be computed with AD.

## Create the discrete model

We consider a $100\times100$ Cartesian mesh of the unit square.

In [ ]:
using Gridap
n = 100
domain = (0,1,0,1)
partition = (n,n)
model = CartesianDiscreteModel(domain,partition)

The default face labeling of a 2D Cartesian discrete model is:

<img src="lexicographic.png" width="240">

We use next these labels to create new Dirichlet tags for the top side and the rest of the boundary.

### Exercise 1

_Create two new boundary tags,`"diri1"` and `"diri0"`, one for the top side of the square and another for the rest of the boundary._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

 Gridap provides a convenient way to create new object identifiers (referred to as "tags") from existing ones. First, we need to extract the object that holds the information about the boundary identifier. Then, we can add new identifiers (aka "tags") to `labels` using `add_tag_from_tags!(labels,"new_tag",[old_tag_1,...,old_tag_n])`.
</details>

In [ ]:
labels = get_face_labeling(model) # Extract boundary identifiers
add_tag_from_tags!(labels,"diri1",[6,])
add_tag_from_tags!(labels,"diri0",[1,2,3,4,5,7,8])

## Set up multifield FE spaces

### Exercise 2

_For the velocities, create a second order vector-valued continuous Lagrangian test FE space. Prescribe Dirichlet BCs._

In [ ]:
D = 2
order = 2
reffeᵤ = ReferenceFE(lagrangian,VectorValue{D,Float64},order)
V_h = TestFESpace(model,reffeᵤ,dirichlet_tags=["diri0","diri1"])

### Exercise 3

_For the pressure, instantiate a linear discontinuous FE space of functions strongly constrained to have zero mean value._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

 - Use `space=:P` to set the reffe in $P_1$.
 - Use `conformity=:L2` to set up a discontinuous FE space.
 - Use `constraint=:zeromean` to set a FE space whose functions have zero mean value.
</details>

In [ ]:
reffeₚ = ReferenceFE(lagrangian,Float64,order-1;space=:P)
Q = TestFESpace(model,reffeₚ,conformity=:L2,constraint=:zeromean)

### Exercise 4

_Create the trial velocity and pressure FE spaces._

<details>
<summary>Click <font color="red"><b>here</b></font> for a hint.</summary>

 Remember to create the functions prescribing the Dirichlet values at the `diri0` and `diri1` regions.
</details>

In [ ]:
uD0 = VectorValue(0,0)
uD1 = VectorValue(1,0)
U = TrialFESpace(V_h,[uD0,uD1])
P = TrialFESpace(Q)

### Multifield FE spaces

In [ ]:
Y = MultiFieldFESpace([V_h, Q])
X = MultiFieldFESpace([U, P])

## Create triangulation and integration measure

In [ ]:
degree = order
Ωₕ = Interior(model)
dΩ = Measure(Ωₕ,degree)

## Nonlinear weak form and FE operator

Multifield FE spaces are Cartesian product FE spaces.

Hence, it's elements are tuples.

We will use `(u,p)` and `(v,q)` for trial and test functions.

### Exercise 5

_Write the bilinear form._

$$
a((u,p),(v,q)) \doteq \int_{\Omega} \nabla u \cdot \nabla v \ {\rm d}\Omega - \int_{\Omega} (\nabla\cdot v) \ p \ {\rm d}\Omega + \int_{\Omega} q \ (\nabla \cdot u) \ {\rm d}\Omega
$$

In [ ]:
a((u,p),(v,q)) = ∫( ∇(u)⊙∇(v) - (∇⋅v)*p + q*(∇⋅u) )dΩ

### Exercise 6

_Write the (nonlinear) convective term._

$$
[c(u)](v) \doteq \int_{\Omega} v \cdot \left( Re \ (u\cdot\nabla)\ u \right)\ {\rm d}\Omega
$$

In [ ]:
const Re = 10.0
c(u,v) = ∫( v⊙(Re*(∇(u)⋅u) ) )dΩ

### Exercise 7

_Write the residual._

In [ ]:
res((u,p),(v,q)) = a((u,p),(v,q)) + c(u,v)

We can already set up the FE operator with the residual.

Then, the Jacobian is computed with AD.

In [ ]:
op = FEOperator(res,X,Y)

## Nonlinear solver phase

Instead of using PETSc, we solve the problem with Gridap's `FESolver`.

In [ ]:
using LineSearches: BackTracking
nls = NLSolver(
  show_trace=true, method=:newton, linesearch=BackTracking())
solver = FESolver(nls)

<details>
<summary>Click <font color="red"><b>here</b></font> for more details.</summary>

 Note that the `NLSolver` function used above internally calls the `nlsolve` function of the [NLsolve](https://github.com/JuliaNLSolvers/NLsolve.jl) package with the provided key-word arguments. Thus, one can use any of the nonlinear methods available via the function `nlsolve` to solve the nonlinear FE problem. Here, we have selected a Newton-Raphson method with a back-tracking line-search from the [LineSearches](https://github.com/JuliaNLSolvers/LineSearches.jl) package.
</details>

We solve the problem without providing an initial guess:

In [ ]:
uh, ph = solve(solver,op) # Default initial guess is zero

Finally, we write the results for visualization.

In [ ]:
writevtk(Ωₕ,"ins-results",cellfields=["uh"=>uh,"ph"=>ph])

<img src="ins_solution.png" width="360">

## References

This exercise has been adapted from the [Gridap Tutorial 8: Incompressible Navier-Stokes](https://gridap.github.io/Tutorials/dev/pages/t008_inc_navier_stokes/)

**Exercise done!**

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*